In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
import nibabel as nib

In [ ]:
##
#Define Path to Code Database
DirPath = '/Your/Path/To/Code/'

##
#Define Output Path
OutputPath = '/Your/Path/To/Output/'

##
#Define Path to HCP1065 MD Map (in FSL this is equivalent to $FSLDIR/data/standard/FSL_HCP1065_MD_1mm.nii.gz)
MDPath = '/usr/local/fsl/data/standard/FSL_HCP1065_MD_1mm.nii.gz'

In [ ]:
##
#Load Functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from ParameterOptionsSimulation import *
from MotionSimulation import *

In [ ]:
##
#Read Parameters
opt = ParameterOptionsSimulation()

##
#Only Model 2 'TRs' (Aka 2 RF Pulses) for the DW-SE sequence
opt['nTR'] = np.asarray([2], dtype='f8')
opt['SteadyStateTR'] = np.asarray([0], dtype='f8')
opt['nDummy'] = np.asarray([0], dtype='f8')

In [ ]:
##
#Define Slice to Sample 
slice = 80

#Load diffusion coefficient map (HCP1065 MD map) & Affine and take single slice
D = np.atleast_3d(nib.load(MDPath).get_fdata())[:,:,slice][:,:,np.newaxis]
Affine = nib.load(MDPath).affine

##
#Get Voxel Dimensions (in mm)
VoxDims = nib.affines.voxel_sizes(Affine)

##
#Create Brain Mask 
Mask = D > 0

##
#Obtain mask indices
MaskIdx = np.where(Mask==1)

##
#Vectorise Diffusion & Mask map for dictionary to accelerate EPG fitting
opt['D'] = np.asarray([*D[Mask==1]], dtype='f8')
opt['Mask'] = np.asarray([*Mask[Mask==1]], dtype='f8')

In [ ]:
##
#Define Sequence
Sequence = 'DW-SE'

##
#Define T1 and T2 values as equal across the simulation
opt['T1'] = opt['Mask']*opt['T1']
opt['T2'] = opt['Mask']*opt['T2']

##
#Define B1 as constant across the image (i.e. no B1 inhomogeneity)
opt['B1'] = np.asarray([*Mask[Mask==1]], dtype='f8') 

In [ ]:
##
#Define Motion Operators for each simulation - Translation Operator (mm/s), Rotation Operator (rad/s) & Cardiac Operator (vector defines maximum beat velocity in mm/s)

##
#Define consistent motion per TR 
RandMotion = False
#Define Heart Rate for Cardiac Operator (beats/min)
HeartRate = 50

##
#Define Operators (Translation Example)
TranslationExample_Trans= np.array([0,0,0.2])
TranslationExample_Rot = np.array([0,0,0])
TranslationExample_Card = np.array([0,0,0])

##
#Define Operators (Rotation Example)
RotationExample_Trans= np.array([0,0,0])
RotationExample_Rot = np.array([0.2/180*np.pi,0,0])
RotationExample_Card = np.array([0,0,0])

##
#Define Operators (Cardiac Example)
CardiacExample_Trans= np.array([0,0,0])
CardiacExample_Rot = np.array([0,0,0])
CardiacExample_Card = np.array([0,0,1.5])


In [ ]:
##
#Simulation - No Motion

##
#Define Output Array
F = np.zeros((*Mask.shape,opt['nTR'].astype(np.int32)[0]), dtype='c8')
#Peform Forward Simulation
F[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt.copy(), Sequence = Sequence)

In [ ]:
##
#Simulation - Translational Motion

##
#Initialise Output Array
FTrans = np.zeros_like(F)

##
#Convert into a Velocity Timeseries Profile
V_Trans = MotionOperator(opt,TranslationExample_Trans,TranslationExample_Rot,TranslationExample_Card,Mask,VoxDims)

##
#Peform Forward Simulation
FTrans[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt.copy(), MotionScale = V_Trans[Mask==1,:], Sequence = Sequence)

In [ ]:
##
#Simulation - Rotational Motion

##
#Define Output Array
FRot = np.zeros_like(F)

##
#Convert into a Velocity Timeseries Profile
V_Rot = MotionOperator(opt,RotationExample_Trans,RotationExample_Rot,RotationExample_Card,Mask,VoxDims)

##
#Peform Forward Simulation
FRot[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt.copy(), MotionScale = V_Rot[Mask==1,:], Sequence = Sequence)

In [ ]:
##
#Simulation - Cardiac Motion

##
#Define Output Array
FCardiac = np.zeros_like(F)

##
#Convert into a Velocity Timeseries Profile
V_Cardiac = MotionOperator(opt,CardiacExample_Trans,CardiacExample_Rot,CardiacExample_Card,Mask,VoxDims,HeartRate=HeartRate)

##
#Peform Forward Simulation
FCardiac[MaskIdx[0],MaskIdx[1],MaskIdx[2],:] = EPGMotionWholeImage(opt.copy(), MotionScale = V_Cardiac[Mask==1,:], Sequence = Sequence)

In [ ]:
##
#Create Figure
fig, axs = plt.subplots(2, 4)
fig.set_size_inches(16,8)

##
#Magnitude Images
axs[0,0].imshow(np.abs(np.rot90(F[15:-20,15:-15,0,-1]))*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*1),clim=[0,0.7],cmap='gray'); axs[0,0].axis('off')
axs[0,1].imshow(np.abs(np.rot90(FTrans[15:-20,15:-15,0,-1]))*np.rot90(Mask[15:-20,15:-15:,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*1),clim=[0,0.7],cmap='gray'); axs[0,1].axis('off')
axs[0,2].imshow(np.abs(np.rot90(FRot[15:-20,15:-15,0,-1]))*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*1),clim=[0,0.7],cmap='gray'); axs[0,2].axis('off')
im = axs[0,3].imshow(np.abs(np.rot90(FCardiac[15:-20,15:-15,0,-1]))*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*1),clim=[0,0.7],cmap='gray'); axs[0,3].axis('off'); plt.colorbar(im,fraction=0.1, pad=0.04, ticks=[0, 0.7],label = 'Magnitude (a.u.)')

##
#Phase Images
axs[1,0].imshow(np.angle(np.rot90(F[15:-20,15:-15,0,-1])*-1)*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*np.pi),clim=[-np.pi/4*3,0],cmap='gray'); axs[1,0].axis('off')
axs[1,1].imshow(np.angle(np.rot90(FTrans[15:-20,15:-15,0,-1])*-1)*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*np.pi),clim=[-np.pi/4*3,0],cmap='gray'); axs[1,1].axis('off')
axs[1,2].imshow(np.angle(np.rot90(FRot[15:-20,15:-15,0,-1])*-1)*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*np.pi),clim=[-np.pi/4*3,0],cmap='gray'); axs[1,2].axis('off')
im = axs[1,3].imshow(np.angle(np.rot90(FCardiac[15:-20,15:-15,0,-1])*-1)*np.rot90(Mask[15:-20,15:-15,0])+np.rot90((Mask[15:-20,15:-15,0] == False)*np.pi),clim=[-np.pi/4*3,0],cmap='gray');  axs[1,3].axis('off'); cbar = plt.colorbar(im,fraction=0.1, pad=0.04, ticks=[-np.pi*3/4, 0, np.pi], label = 'Phase (rad.)');  cbar.set_ticklabels([r'-$\frac{3}{4}\cdot\pi$',0, r'$\pi$'])

##
#Additional Text and Whitespace Edits
axs[0,0].text(-0, 1.05, '(a) No Motion', transform=axs[0,0].transAxes, size=15)
axs[0,1].text(-0, 1.05, '(b) Translation', transform=axs[0,1].transAxes, size=15)
axs[0,2].text(-0, 1.05, '(c) Rotation', transform=axs[0,2].transAxes, size=15)
axs[0,3].text(-0, 1.05, '(d) Pulsatility', transform=axs[0,3].transAxes, size=15)
fig.subplots_adjust(hspace=0.05,wspace=-0.5)

In [ ]:
##
#Save Figure
fig.savefig(''.join([OutputPath,'FigureS3.png']),dpi=300,format='png',bbox_inches='tight')